In [483]:
import pandas as pd
import os
import datetime
import shutil
pd.options.mode.chained_assignment = None

## Importation of datasets

In [484]:
folder_path = r"C:\Users\Samuel\Documents\pascal"

file_names = []     
file_count = 0
    
for subdir, _, files in os.walk(folder_path):
    files.sort() #sort files

    for filename in files:
        if filename.startswith('covid_help_data') is True or filename.startswith('worldwide') is True:
            file_count+=1

            filepath = subdir + os.sep + filename

            if filepath.endswith(".csv"):
                file_names.append(filepath)
        else:
            file_count+=1
    
print("There are " + str(file_count) + " total files")   
print("There are " + str(len(file_names)) + " needed csv files")

There are 182 total files
There are 181 needed csv files


# Task 1 solution

### Gouping of files

In [485]:
covid_file_names = [i for i in file_names if 'covid_help_data' in i]
world_file_names = [i for i in file_names if 'worldwide' in i]
print(f"covid* total files: {len(covid_file_names)}")
print(f"world* total files: {len(world_file_names)}")

covid* total files: 93
world* total files: 88


In [486]:
print(covid_file_names[0])
print(covid_file_names[0][-12:-4])

C:\Users\Samuel\Documents\pascal\Group 1\covid_help_data_01-May-2020 23-40-36.csv
23-40-36


In [487]:
# Extraction of timestamps for both groups
covid_time = {i:datetime.datetime.strptime(i[-12:-4], '%H-%M-%S').time() for i in covid_file_names}
world_time = {i:datetime.datetime.strptime(i[-12:-4], '%H-%M-%S').time() for i in world_file_names}

## Legend
* **Group 1:** covid* 11pm+
* **Group 2:** world* 11pm+
* **Group 3:** covid* other time
* **Group 4:** world* other time

### Separation of files into 4 groups

In [488]:
group_1_file_paths, group_2_file_paths, group_3_file_paths, group_4_file_paths = [], [], [], []

for i, j in zip(covid_time.keys(), covid_time.values()):
    if j > datetime.datetime.strptime('23-00-00', '%H-%M-%S').time():
        group_1_file_paths.append(i)
    else:
        group_3_file_paths.append(i)

print(f"Group 1 total files: {len(group_1_file_paths)}")
print(f"Group 3 total files: {len(group_3_file_paths)}")

print()

for i, j in zip(world_time.keys(), world_time.values()):
    if j > datetime.datetime.strptime('23-00-00', '%H-%M-%S').time():
        group_2_file_paths.append(i)
    else:
        group_4_file_paths.append(i)

print(f"Group 2 total files: {len(group_2_file_paths)}")
print(f"Group 4 total files: {len(group_4_file_paths)}")

Group 1 total files: 42
Group 3 total files: 51

Group 2 total files: 43
Group 4 total files: 45


In [489]:
# Extraction of only the filenames of the dataset per groups
group_1_file_names, group_2_file_names, group_3_file_names, group_4_file_names = [i[i.rfind('\\')+1:] for i in group_1_file_paths], [i[i.rfind('\\')+1:] for i in group_2_file_paths], [i[i.rfind('\\')+1:] for i in group_3_file_paths], [i[i.rfind('\\')+1:] for i in group_4_file_paths]
print(group_1_file_names[0])
print(group_2_file_names[0])
print(group_3_file_names[0])
print(group_4_file_names[0])

covid_help_data_01-May-2020 23-40-36.csv
worldwide_01-May-2020 23-49-16.csv
covid_help_data_01-May-2020 10-02-27.csv
worldwide_01-May-2020 10-11-07.csv


### Creation of a directory per group and moving of each datsets per group into their respective directories locally

In [490]:
# for i in range(1, 5):
#     os.mkdir(f'C:\\Users\\Samuel\\Documents\\pascal\\Group {i}')

os.chdir(r'C:\Users\Samuel\Documents\pascal')
print(os.getcwd())

# for i in group_1_file_names:
#     shutil.move(i, 'Group 1')

# for i in group_2_file_names:
#     shutil.move(i, 'Group 2')
    
# for i in group_3_file_names:
#     shutil.move(i, 'Group 3')
    
# for i in group_4_file_names:
#     shutil.move(i, 'Group 4')

C:\Users\Samuel\Documents\pascal


# Task 2 solution
**Question:** Using Group 2, how many cases did the US and Germany each record in April?

### Extraction of Group 2 datasets for the month of April

In [491]:
APRIL_group_2_file_names = [i for i in group_2_file_names if 'Apr' in i]
print(len(APRIL_group_2_file_names))

print()

print(APRIL_group_2_file_names[0])

print()

os.chdir(r'C:\Users\Samuel\Documents\pascal\Group 2')
print(os.getcwd())

print()

APRIL_group_2 = [pd.read_csv(i) for i in APRIL_group_2_file_names]
print(len(APRIL_group_2))

print()

APRIL_group_2_lengths = [len(i) for i in APRIL_group_2]
print(APRIL_group_2_lengths)

print()

print(sum(APRIL_group_2_lengths))

19

worldwide_02-Apr-2020 23-56-11.csv

C:\Users\Samuel\Documents\pascal\Group 2

19

[206, 206, 206, 188, 190, 190, 197, 199, 200, 200, 200, 203, 205, 214, 214, 214, 214, 214, 216]

3876


### Date-time dataframe to be appended appropriately to the last column in each dataset

In [492]:
prefix = 'worldwide_'
date = '**-***-****'

date_time = {i[len(prefix):len(prefix)+len(date)]:i[-12:-4] for i in APRIL_group_2_file_names}

dates = [i for i, j in zip(date_time.keys(), APRIL_group_2_lengths) for _ in range(0, j)]       
date_dict = {str(i):j for i, j in zip(range(0, len(dates)), dates)}

times = [i for i, j in zip(date_time.values(), APRIL_group_2_lengths) for _ in range(0, j)]       
time_dict = {str(i):j for i, j in zip(range(0, len(times)), times)}

date_time_df = pd.concat([pd.Series(date_dict), pd.Series(time_dict)], axis = 1)
date_time_df.rename(columns = {0:'Date', 1:'Time'}, inplace = True)
print(len(date_time_df))

3876


### Concating all datasets present in the group
`tail-to-head` operation. 

In [493]:
APRIL_group_2_df = pd.concat(APRIL_group_2)
APRIL_group_2_df.columns.tolist()

['Country',
 'Total confirmed cases',
 'Total deaths',
 'Confirmed recoveries',
 'Cases confirmed today',
 'Deaths today',
 'Recoveries confirmed today',
 'Total confirmed',
 'Total_confirmed_today',
 'Total_deaths',
 'Total_deaths_today',
 'Total_Recoveries',
 'Total_Recoveries_today',
 'Active_confirmed',
 'Critical',
 'Mortality_close',
 'Mortality_confirmed',
 'Total_tests',
 'Total_tests_today']

### Date-time dataframe is added to the last column of the concated dataframe

In [494]:
APRIL_group_2_df['Date'] = date_time_df.Date.values
APRIL_group_2_df['Time'] = date_time_df.Time.values
APRIL_group_2_df.shape

(3876, 21)

### Retrieving countries that starts with letters *u* and *g* respectively.
**N.B:** This is done becuase some country names could be abbreviated. For example, `US`, `UK`

In [495]:
u_variants = sorted([i for i in set(APRIL_group_2_df.Country.values) if i.startswith('U') is True or i.startswith('u') is True])
g_variants = sorted([i for i in set(APRIL_group_2_df.Country.values) if i.startswith('G') is True or i.startswith('g') is True])
var = {'U_countries':u_variants, 'G_countries':g_variants}
var

{'U_countries': ['USA',
  'uganda',
  'ukraine',
  'united-arab-emirates',
  'united-kingdom',
  'united-states',
  'uruguay',
  'uzbekistan'],
 'G_countries': ['gabon',
  'gambia',
  'georgia',
  'germany',
  'ghana',
  'gibraltar',
  'greece',
  'greenland',
  'grenada',
  'guadeloupe',
  'guatemala',
  'guinea',
  'guinea-bissau',
  'guyana']}

### Data preprocessing

In [496]:
cols = ['Country', 'Total confirmed cases', 'Cases confirmed today', 'Total confirmed', 'Total_confirmed_today', 'Date', 'Time']
APRIL_group_2_df_valid = APRIL_group_2_df[cols].sort_values(by = 'Country')
APRIL_group_2_df_valid.reset_index(drop = True)

,Country,Total confirmed cases,Cases confirmed today,Total confirmed,Total_confirmed_today,Date,Time
0,USA,NaN,NaN,22,1,07-Apr-2020,23-49-27
1,USA,NaN,NaN,172,14,07-Apr-2020,23-49-27
2,USA,NaN,NaN,171,7,07-Apr-2020,23-49-27
3,USA,NaN,NaN,170,25,07-Apr-2020,23-49-27
4,USA,NaN,NaN,166,1,07-Apr-2020,23-49-27
...,...,...,...,...,...,...,...
3871,zimbabwe,9,0,NaN,NaN,03-Apr-2020,23-40-26
3872,zimbabwe,NaN,NaN,25,1,19-Apr-2020,23-49-16
3873,zimbabwe,NaN,NaN,23,6,16-Apr-2020,23-49-31
3874,zimbabwe,NaN,NaN,28,3,22-Apr-2020,23-49-01


### Data processing

In [497]:
APRIL_group_2_df_valid.fillna(0, inplace = True)

APRIL_group_2_df_valid['Cases confirmed today'] = APRIL_group_2_df_valid['Cases confirmed today'].astype(str)
APRIL_group_2_df_valid['Total_confirmed_today'] = APRIL_group_2_df_valid['Total_confirmed_today'].astype(str)
APRIL_group_2_df_valid['Total confirmed cases'] = APRIL_group_2_df_valid['Total confirmed cases'].astype(str)
APRIL_group_2_df_valid['Total confirmed'] = APRIL_group_2_df_valid['Total confirmed'].astype(str)

APRIL_group_2_df_valid['Cases confirmed today'] = APRIL_group_2_df_valid['Cases confirmed today'].apply(lambda x: x.replace(',', ''))
APRIL_group_2_df_valid['Total_confirmed_today'] = APRIL_group_2_df_valid['Total_confirmed_today'].apply(lambda x: x.replace(',', ''))
APRIL_group_2_df_valid['Total confirmed cases'] = APRIL_group_2_df_valid['Total confirmed cases'].apply(lambda x: x.replace(',', ''))
APRIL_group_2_df_valid['Total confirmed'] = APRIL_group_2_df_valid['Total confirmed'].apply(lambda x: x.replace(',', ''))

APRIL_group_2_df_valid['Cases confirmed today'] = APRIL_group_2_df_valid['Cases confirmed today'].astype(int)
APRIL_group_2_df_valid['Total_confirmed_today'] = APRIL_group_2_df_valid['Total_confirmed_today'].astype(int)
APRIL_group_2_df_valid['Total confirmed cases'] = APRIL_group_2_df_valid['Total confirmed cases'].astype(int)
APRIL_group_2_df_valid['Total confirmed'] = APRIL_group_2_df_valid['Total confirmed'].astype(int)


APRIL_group_2_df_valid['Total_cases_today'] = APRIL_group_2_df_valid['Cases confirmed today'] + APRIL_group_2_df_valid['Total_confirmed_today']
APRIL_group_2_df_valid['Cumulative_total_cases'] = APRIL_group_2_df_valid['Total confirmed cases'] + APRIL_group_2_df_valid['Total confirmed']

APRIL_group_2_df_valid_final = APRIL_group_2_df_valid[['Country', 'Total_cases_today', 'Cumulative_total_cases', 'Date', 'Time']]

# This confirms that the different column names for each similar records across the datasets concated maintains the data integrity
print(len(APRIL_group_2_df_valid.loc[((APRIL_group_2_df_valid['Cases confirmed today'] > 0) & (APRIL_group_2_df_valid['Total_confirmed_today'] > 0))]))
print(len(APRIL_group_2_df_valid.loc[((APRIL_group_2_df_valid['Total confirmed cases'] > 0) & (APRIL_group_2_df_valid['Total confirmed'] > 0))]))

0
0


### Data split per country

In [498]:
US = APRIL_group_2_df_valid_final.loc[((APRIL_group_2_df_valid_final.Country == 'USA') | (APRIL_group_2_df_valid_final.Country == 'united-states'))]
germany = APRIL_group_2_df_valid_final[APRIL_group_2_df_valid_final['Country'] == 'germany']

print(len(US))
print(len(germany))

584
16


### Analysis result

In [499]:
print(f"Total US cases in April: {(US['Cumulative_total_cases'].loc[US.Date == dates[-1]].item()) - (US['Cumulative_total_cases'].loc[US.Date == dates[0]].item()):,}")
print(f"Total Germany cases in April: {(germany['Cumulative_total_cases'].loc[germany.Date == dates[-1]].item()) - (germany['Cumulative_total_cases'].loc[germany.Date == dates[0]].item()):,}")

# print()

# print(f"Total US cases per day in April: {US['Total_cases_today'].sum():,}")
# print(f"Total Germany per day cases in April: {germany['Total_cases_today'].sum():,}")

Total US cases in April: 851,822
Total Germany cases in April: 78,215


**NB:** *The flow of approach for solving task 3 as illustrated below is exactly similar to that of task 2 above. Hence, no repetition of markdown cells, comments*.

# Task 3 solution
**Question:** Using Group 2, how many cases did the US and Germany each record in April?

In [500]:
APRIL_group_3_file_names = [i for i in group_3_file_names if 'Apr' in i]
print(len(APRIL_group_3_file_names))

print()

print(APRIL_group_3_file_names[0])

print()

os.chdir(r'C:\Users\Samuel\Documents\pascal\Group 3')
print(os.getcwd())

print()

APRIL_group_3 = [pd.read_csv(i) for i in APRIL_group_3_file_names]
print(len(APRIL_group_3))

print()

APRIL_group_3_lengths = [len(i) for i in APRIL_group_3]
print(APRIL_group_3_lengths)

print()

print(sum(APRIL_group_3_lengths))

28

covid_help_data_03-Apr-2020 00-40-28.csv

C:\Users\Samuel\Documents\pascal\Group 3

28

[209, 209, 209, 210, 212, 212, 212, 218, 213, 213, 213, 213, 213, 213, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214]

5965


In [501]:
prefix_a = 'covid_help_data_'
date_a = '**-***-****'

date_1 = [i[len(prefix_a):len(prefix_a)+len(date_a)] for i in APRIL_group_3_file_names]
time_1 = [i[-12:-4] for i in APRIL_group_3_file_names]

dates_a = [i for i, j in zip(date_1, APRIL_group_3_lengths) for _ in range(0, j)]       
date_dict_a = {str(i):j for i, j in zip(range(0, len(dates_a)), dates_a)}

times_a = [i for i, j in zip(time_1, APRIL_group_3_lengths) for _ in range(0, j)]       
time_dict_a = {str(i):j for i, j in zip(range(0, len(times_a)), times_a)}

date_time_df_a = pd.concat([pd.Series(date_dict_a), pd.Series(time_dict_a)], axis = 1)
date_time_df_a.rename(columns = {0:'Date', 1:'Time'}, inplace = True)
print(len(date_time_df_a))

5965


In [502]:
APRIL_group_3_df = pd.concat(APRIL_group_3)
APRIL_group_3_df.columns.tolist()

['countries',
 'infected',
 'deaths',
 'recovered',
 'active',
 'death_rate',
 'recovery_rate',
 'infected_today',
 'deaths_today',
 'recovered_today',
 'critical',
 'tests']

In [503]:
APRIL_group_3_df['Date'] = date_time_df_a.Date.values
APRIL_group_3_df['Time'] = date_time_df_a.Time.values
APRIL_group_3_df.shape

(5965, 14)

In [504]:
i_variants = sorted([i for i in set(APRIL_group_3_df.countries.values) if i.startswith('I') is True or i.startswith('i') is True])
r_variants = sorted([i for i in set(APRIL_group_3_df.countries.values) if i.startswith('R') is True or i.startswith('r') is True])
vars_ = {'I_countries':i_variants, 'R_countries':r_variants}
vars_

{'I_countries': ['Iceland',
  'India',
  'Indonesia',
  'Iran',
  'Iraq',
  'Ireland',
  'Isle of Man',
  'Israel',
  'Italy',
  'Ivory Coast'],
 'R_countries': ['Romania', 'Russia', 'Rwanda', 'Réunion']}

In [505]:
cols = ['countries', 'infected', 'infected_today', 'Date', 'Time']
APRIL_group_3_df_valid = APRIL_group_3_df[cols].sort_values(by = 'countries')
APRIL_group_3_df_valid.reset_index(drop = True)

,countries,infected,infected_today,Date,Time
0,Afghanistan,281,NaN,04-Apr-2020,00-40-27
1,Afghanistan,1026,30.0,20-Apr-2020,10-02-25
2,Afghanistan,607,52.0,12-Apr-2020,14-58-08
3,Afghanistan,444,0.0,09-Apr-2020,06-40-42
4,Afghanistan,1092,66.0,21-Apr-2020,10-02-27
...,...,...,...,...,...
5960,Zimbabwe,24,0.0,18-Apr-2020,10-02-21
5961,Zimbabwe,40,8.0,30-Apr-2020,10-02-28
5962,Zimbabwe,25,0.0,20-Apr-2020,10-02-25
5963,Zimbabwe,10,0.0,07-Apr-2020,14-39-19


In [506]:
APRIL_group_3_df_valid.fillna(0, inplace = True)

APRIL_group_3_df_valid['infected_today'] = APRIL_group_3_df_valid['infected_today'].astype(int)

italy = APRIL_group_3_df_valid[APRIL_group_3_df_valid['countries'] == 'Italy']
russia = APRIL_group_3_df_valid[APRIL_group_3_df_valid['countries'] == 'Russia']

print(len(italy))
print(len(russia))

28
28


### Analysis result

In [507]:
print(f"Total Italy cases in April: {(italy['infected'].loc[italy.Date == dates_a[-1]].item()) - (italy['infected'].loc[italy.Date == dates_a[0]].item()):,}")
print(f"Total Russia cases in April: {(russia['infected'].loc[russia.Date == dates_a[-1]].item()) - (russia['infected'].loc[russia.Date == dates_a[0]].item()):,}")

# print()

# print(f"Total Italy per cases today in April: {italy['infected_today'].sum():,}")
# print(f"Total Russia per cases today in April: {russia['infected_today'].sum():,}")

Total Italy cases in April: 88,349
Total Russia cases in April: 102,950
